# Probability

An intro to **probability** in Clojure

## `P` function

`P` is the name for the *Probability* function

In [10]:
(ns probability
  (:require [clojure.math.combinatorics :as combinatorics]))

In [11]:
(defn P
  "Probability of an event occurring"
  [event space]
  (/ (count (filter (set event) space))
     (count space)))

#'probability/P

In [12]:
(def D [1 2 3 4 5 6])
(def even [2 4 6])

#'probability/even

In [13]:
(P even D)

1/2

We might have done `(/ (count event) (count space))`, but in this way we would have counted stuff not present in the event space.

In [14]:
(def even [2 4 6 8 10 12])

(P even D)

1/2

## Urn problems

Usually urns are used to explain probability. Let's take this problem as an example:

> An urn contains 23 balls: 8 white, 6 blue, and 9 red. We select six balls at random (each possible selection is equally likely). What is the probability of each of these possible outcomes:
1. all balls are red
2. 3 are blue, 2 are white, and 1 is red
3. exactly 4 balls are white

Here an *outcome* is a set of 6 balls and the *sample space* is the set of all possible 6 ball combinations. We can solve this problem with our `P` function and *counting*.

Note that an outcome is a *set* of balls, not a *sequence*, so the order doesn't matter. To account for the fact that we have more than one ball of the same color, we will represent them with the initial followed by a number, such as: `W1`

In [15]:
(defn cross
  "Create a set of ways of concatenating
  items from A with items from B"
  [A B]
  (set (map #(str A %) B)))

#'probability/cross

In [16]:
(def balls [{:A "W" :B (range 1 9)}
            {:A "B" :B (range 1 7)}
            {:A "R" :B (range 1 10)}])

#'probability/balls

In [17]:
(def urn (mapcat #(cross (:A %) (:B %)) balls))

#'probability/urn

In [18]:
(count urn)

23

By using `clojure.math.combinatorics` (here required in the `ns` declaration in the first cell as `combinatorics`) we can create the sample space as the set of all 6 balls combinations.

In [19]:
(defn combos
  "All combinations of n items, each one concatenated
  with str"
  [items n]
  (let [combi (combinatorics/combinations items n)]
    (map #(clojure.string/join " " %) combi)))

#'probability/combos

In [20]:
(def U6 (combos urn 6))

(count U6)

100947

In [21]:
(clojure.pprint/pprint (take 10 (random-sample 0.5 U6)))

("W3 W7 W6 W1 W8 W5"
 "W3 W7 W6 W1 W8 W4"
 "W3 W7 W6 W1 W8 W2"
 "W3 W7 W6 W1 W8 B5"
 "W3 W7 W6 W1 W8 B4"
 "W3 W7 W6 W1 W8 R6"
 "W3 W7 W6 W1 W8 R1"
 "W3 W7 W6 W1 W5 W4"
 "W3 W7 W6 W1 W5 W2"
 "W3 W7 W6 W1 W5 B1")


To check if it is true that 100,947 is the right number of possible combinations we can build a function that calculates it

In [22]:
(defn ! [n]
  (reduce *' 1 (range 1 (inc n))))

(defn choose
  "Number of ways to choose c items from
  a list of n items"
  [n c]
  (int (/ (! n) 
          (* (! (- n c)) (! c)))))

#'probability/choose

In [23]:
(choose 23 6)

100947

### Problem 1: what's the probability of selecting 6 red balls?

In [24]:
(defn count-color
  [string color]
  (->> string
       (re-seq (re-pattern color))
       count))

(defn draw
  [urn color n]
  (filter 
    #(= (count-color % color) n) 
    urn))

(def red6 (draw U6 "R" 6))

#'probability/red6

In [25]:
(P red6 U6)

4/4807

In [26]:
(count red6)

84

In [27]:
(choose 9 6)

84

As we saw there are **84** different ways of drawing 6 red balls. Since there are 9 red balls in the urn, we are asking `(choose 9 6)` basically. This means that **P** of 6 red balls is 9 choose 6 divided by the size of the sample space.

In [28]:
(= (P red6 U6)
   (/ (choose 9 6)
      (count U6)))

true

### Problem 2: what is the probability of 3 blue, 2 white and 1 red?

In [29]:
(defn draw
  [urn colors n]
  (let [count-colors (fn [s c]
                       (map #(count-color s %) c))
        right-draw? (fn [s c n]
                      (= (count-colors s c)
                         n))]
    (filter #(right-draw? % colors n) urn)))

#'probability/draw

In [30]:
(def b3w2r1 (draw U6 ["B" "W" "R"] [3 2 1]))

(P b3w2r1 U6)

240/4807

In [31]:
(= (P b3w2r1 U6)
   (/ (* (choose 6 3)
         (choose 8 2)
         (choose 9 1))
      (count U6)))

true

### Problem 3: what is the probability of drawing 4 white balls?

In [32]:
(def w4 (draw U6 ["W"] [4]))

(P w4 U6)

350/4807

## Generalizing `P`

In [33]:
(defn P
  [event space]
  (let [e (if (fn? event)
            (filter event space)
            (filter (set event) space))]
    (/ (count e)
       (count space))))

#'probability/P

In [34]:
(P even? D)

1/2

In [35]:
(def D12 (range 1 13))

(P even? D12)

1/2

In [36]:
(defn divisible? 
  [a b]
  (zero? (mod a b)))

(defn prime? 
  [n]
  (and (> n 1) (not-any? (partial divisible? n) (range 2 n))))

#'probability/prime?

In [37]:
(def D3 
  (->> (for [a D
             b D
             c D]
         [a b c])
       (map #(reduce + %))))

#'probability/D3

In [38]:
(P prime? D3)

73/216

## Card Problems

Let's play some Poker! We define a `deck` as a set of 52 cards, and `hands` as the sample space of all combinations of 5 cards:

In [39]:
(def ranks (clojure.string/split "A23456789TJQK" #""))

(def suits&ranks
  [{:A "S" :B ranks}
   {:A "H" :B ranks}
   {:A "D" :B ranks}
   {:A "C" :B ranks}])

#'probability/suits&ranks

In [40]:
(def deck (atom 
            (mapcat #(cross (:A %) (:B %)) suits&ranks)))

#'probability/deck

In [41]:
(count @deck)

52

In [42]:
(def hands (atom (combos @deck 5)))

#'probability/hands

In [43]:
(= (count @hands)
   (choose 52 5))

true

In [44]:
(count @hands)

2598960

In [45]:
(clojure.pprint/pprint 
  (take 10 (random-sample 0.5 @hands)))

("S9 S6 S2 S5 S8"
 "S9 S6 S2 S5 SQ"
 "S9 S6 S2 S5 S3"
 "S9 S6 S2 S5 H3"
 "S9 S6 S2 S5 HT"
 "S9 S6 S2 S5 H4"
 "S9 S6 S2 S5 H2"
 "S9 S6 S2 S5 H7"
 "S9 S6 S2 S5 D3"
 "S9 S6 S2 S5 D9")


In [46]:
(defn flush?
  [hand]
  (let [suits (map :A suits&ranks)]
    (some true?
          (map #(= (count-color hand %) 5) suits))))

#'probability/flush?

In [47]:
(P flush? @hands)

33/16660

In [48]:
(defn four-kind?
  [hand]
  (some true?
        (map #(= (count-color hand %) 4) ranks)))

#'probability/four-kind?

In [49]:
(P four-kind? @hands)

1/4165

## Gambling, Triangles and the birth of Probability

In [50]:
(defn continuations
  [H T]
  (let [rounds (map #(clojure.string/split % #"") (repeat (- (+ H T) 1) "ht"))]
    (apply combinatorics/cartesian-product rounds)))

#'probability/continuations

In [51]:
(defn win-probability
  [H T]
  (let [hwins (fn [out]
                (>= (count
                      (filter
                        #(= "h" %) out)) H))]
   (P hwins (continuations H T))))

#'probability/win-probability

In [52]:
(continuations 2 3)

(("h" "h" "h" "h") ("h" "h" "h" "t") ("h" "h" "t" "h") ("h" "h" "t" "t") ("h" "t" "h" "h") ("h" "t" "h" "t") ("h" "t" "t" "h") ("h" "t" "t" "t") ("t" "h" "h" "h") ("t" "h" "h" "t") ("t" "h" "t" "h") ("t" "h" "t" "t") ("t" "t" "h" "h") ("t" "t" "h" "t") ("t" "t" "t" "h") ("t" "t" "t" "t"))

In [53]:
(win-probability 2 3)

11/16

## Probability Distributions

In [54]:
(defn make-prob-dist
  [probs]   
  (let [values (vals probs)
        total  (apply + values)
        dist (zipmap 
               (keys probs) 
               (map #(/ % total) values))]
    (assert (every? #(>= % 0) (vals dist)))
    dist))

#'probability/make-prob-dist

In [55]:
(defn P
  [event space]
  (cond
    (and (fn? event)
         (map? space)) (->> (filter event (keys space))
                            (select-keys space)
                            vals
                            (reduce +))
    (fn? event) (/ (count (filter event space))
                   (count space))
    :else (/ (count (filter (set event) space))
             (count space))))

#'probability/P

In [56]:
(def DK
  (make-prob-dist {"GG" 121801
                   "GB" 126840
                   "BG" 127123
                   "BB" 135138}))

DK

{"GG" 121801/510902, "GB" 9060/36493, "BG" 127123/510902, "BB" 67569/255451}

In [57]:
(defn make-pred
  [pos sex]
  (let [pos (if (= 0 pos)
              [0 1]
              [pos])]
    (fn [k] 
      (= (apply subs k pos) sex))))

#'probability/make-pred

In [58]:
(defn filter-dist
  [pred dist]
  (let [k (filter pred (keys dist))]
    (make-prob-dist (select-keys dist k))))

#'probability/filter-dist

In [59]:
(def first-girl?
  (make-pred 0 "G"))

#'probability/first-girl?

In [60]:
(P first-girl? DK)

248641/510902

In [61]:
(P (make-pred 0 "G") DK)

248641/510902

In [62]:
(filter-dist (make-pred 0 "B") DK)

{"BG" 127123/262261, "BB" 135138/262261}

In [63]:
(P (make-pred 1 "G") (filter-dist first-girl? DK))

121801/248641

In [64]:
(P (make-pred 1 "G") (filter-dist (make-pred 0 "B") DK))

127123/262261

In [65]:
(P (make-pred 1 "B") (filter-dist first-girl? DK))

126840/248641

In [66]:
(P (make-pred 1 "B") (filter-dist (make-pred 0 "B") DK))

135138/262261

## M&Ms & Bayes

In [67]:
(def bag-94
  (make-prob-dist {"brown" 30
                   "yellow" 20
                   "red" 20
                   "green" 10
                   "orange" 10
                   "tan" 10}))

(def bag-96
  (make-prob-dist {"blue" 24
                   "yellow" 14
                   "red" 13
                   "green" 20
                   "orange" 16
                   "brown" 13}))

#'probability/bag-96

In [68]:
(defn joint
  [A B]
  (let [k (clojure.math.combinatorics/cartesian-product (keys A) (keys B))
        v (clojure.math.combinatorics/cartesian-product (vals A) (vals B))
        nk (map #(clojure.string/join #" " %) k)
        nv (map #(apply * %) v)
        res (zipmap nk nv)]
    (make-prob-dist res)))

#'probability/joint

In [69]:
(clojure.pprint/pprint (joint bag-94 bag-96))

{"green brown" 13/1000,
 "tan orange" 2/125,
 "red orange" 4/125,
 "orange red" 13/1000,
 "green yellow" 7/500,
 "brown orange" 6/125,
 "orange blue" 3/125,
 "tan brown" 13/1000,
 "tan yellow" 7/500,
 "orange brown" 13/1000,
 "yellow red" 13/500,
 "yellow brown" 13/500,
 "brown yellow" 21/500,
 "red brown" 13/500,
 "tan blue" 3/125,
 "yellow yellow" 7/250,
 "brown brown" 39/1000,
 "yellow orange" 4/125,
 "orange orange" 2/125,
 "orange yellow" 7/500,
 "orange green" 1/50,
 "brown blue" 9/125,
 "brown red" 39/1000,
 "red blue" 6/125,
 "yellow green" 1/25,
 "green orange" 2/125,
 "red green" 1/25,
 "green red" 13/1000,
 "green blue" 3/125,
 "red yellow" 7/250,
 "yellow blue" 6/125,
 "brown green" 3/50,
 "red red" 13/500,
 "tan red" 13/1000,
 "green green" 1/50,
 "tan green" 1/50}


In [70]:
(def MM (joint bag-94 bag-96))

#'probability/MM

In [71]:
(require '[clojure.string :refer [includes? starts-with?]])

(def yellow-and-green
  (fn [k]
    (and
      (includes? k "yellow")
      (includes? k "green"))))

#'probability/yellow-and-green

In [72]:
(filter-dist yellow-and-green MM)

{"green yellow" 7/27, "yellow green" 20/27}

In [73]:
(defn yellow94
  [k]
  (starts-with? k "yellow"))

#'probability/yellow94

In [74]:
(P yellow94 (filter-dist yellow-and-green MM))

20/27

## Newton's Answer to a Problem by Pepys

In [75]:
(def die
  (make-prob-dist {"6" 1/6
                   "-" 5/6}))

#'probability/die

In [76]:
(defn dice
  [n die]
  (if (= n 1)
    die
    (joint die (dice (- n 1) die))))

#'probability/dice

In [77]:
(dice 3 die)

{"6 6 6" 1/216, "6 6 -" 5/216, "6 - 6" 5/216, "6 - -" 25/216, "- 6 6" 5/216, "- 6 -" 25/216, "- - 6" 25/216, "- - -" 125/216}

In [78]:
(defn at-least
  [k res]
  (fn [s]
    (>= (count (re-seq (re-pattern res) s)) k)))

#'probability/at-least

In [79]:
(P (at-least 1 "6") (dice 6 die))

31031/46656

In [80]:
(P (at-least 2 "6") (dice 12 die))

1346704211/2176782336

In [81]:
(P (at-least 3 "6") (dice 18 die))

15166600495229/25389989167104

---------------------------

# Simulation

## Monopoly

In [82]:
(defn split-on-space [word] 
  (clojure.string/split word #"\s"))

(def board
  (->> "GO   A1 CC1 A2  T1 R1 B1  CH1 B2 B3
   JAIL C1 U1  C2  C3 R2 D1  CC2 D2 D3 
   FP   E1 CH2 E2  E3 R3 F1  F2  U2 F3 
   G2J  G1 G2  CC3 G3 R4 CH3 H1  T2 H2"
       split-on-space 
       (filter #(not (clojure.string/blank? %)))))

#'probability/board

In [85]:
(cross board)

ArityException Wrong number of args (1) passed to: probability/cross  clojure.lang.AFn.throwArity (AFn.java:429)


class clojure.lang.ArityException: 

In [84]:
(defn monopoly
  [steps]
  )

#'probability/monopoly